In [12]:
import requests
import bs4
import re
import json

In [13]:
documents = [
    '25',
    '261',
    '306',
    '313',
    '352',
    '359',
    '360',
    '381',
    '603',
    '654',
    '655',
    '656',
    '662',
    '664',
    '677',
    '683',
    '714',
    '724',
    '749',
    '783'
]

In [14]:
def custom_join(soup):
    text_parts = []
    for element in soup.descendants:
        if element.name == 'em':
            # Добавляем текст внутри <em> без переноса строки
            if element.string:
                text_parts.append(' ' + element.string)
        elif isinstance(element, str):
            # Добавляем текстовые ноды с переносом строки, если они не внутри <em>
            stripped_string = element.strip()
            if stripped_string:
                # Проверяем, не является ли предыдущий элемент <em> для избежания лишних переносов
                if text_parts and text_parts[-1].endswith('\n'):
                    text_parts[-1] += stripped_string
                else:
                    text_parts.append(stripped_string + '\n')
        elif element.name in ['p', 'div']:
            # Явное добавление переноса строки после параграфов и дивов
            text_parts.append('\n')

    return ''.join(text_parts)

In [15]:
def clean_text(text):
    # Регулярное выражение для поиска паттернов с учетом неразрывного пробела
    pattern = r'\(J\d+(\.\d+)?\)|J\d+(\.\d+)?'
    # Замена найденных паттернов на пустую строку
    cleaned_text = re.sub(pattern, '', text)
    return cleaned_text

In [16]:
def find_by_element_by_id(data,id):
    for i in range(len(data)):
        if(data[i]['id']==id):
            return i
    return None

In [17]:
def find_diseases_in_content(data):
    content = data[find_by_element_by_id(data,'doc_1')]['content']
    soup = bs4.BeautifulSoup(content)
    related_diseases = 'Особенности кодирования заболевания'.lower()

    current_element = None
    for h in soup.findAll('h2'):
        if(related_diseases in h.text.lower()):
            current_element = h
            break
    
    current_element = current_element.findNext()
    text = ''
    while(current_element.name != 'h2'):
        text += f'{current_element.text}\n'
        current_element = current_element.findNext()
    return text

In [33]:
def parse_document(id):
    url = f'https://apicr.minzdrav.gov.ru/api.ashx?op=GetClinrec2&id={id}&ssid=undefined'
    request = requests.get(url)

    data = request.json()['obj']['sections']

    start = 'Диагностика'.lower()
    end = 'Дополнительная информация'.lower()
    title = 'Титульный лист'.lower()

    start_index = None
    end_index = None
    title_index = None

    for i in range(len(data)):
        d = data[i]
        name = d['title'].lower()
        if(start in name):
            start_index = i
        if(end in name):
            end_index = i
        if(title in name):
            title_index = i

        if(end_index != None and start_index != None and title_index != None):
            break
    
    text = ''
    for i in range(start_index,end_index+1):
        d = data[i]
        title = d['title']

        soup = bs4.BeautifulSoup(d['content'])
        content = ' \n'.join(soup.stripped_strings)
        text += f'''
        {title}\n
        {content}\n
        '''

    related_diseases_text = ''

    related_diseases_index = find_by_element_by_id(data,'doc_crat_info_1_4')

    if(related_diseases_index!=None):
        d = data[related_diseases_index]
        soup = bs4.BeautifulSoup(d['content'])
        related_diseases_text = soup.getText()
    else:
        print('Related text was not found. Using content')
        related_diseases_text = find_diseases_in_content(data)


    d = data[title_index]
    main_disease = d['data'][0]['content']

    related_diseases_text = clean_text(related_diseases_text)

    cr_info = f'''
<id>{id}</id>:
<title>{main_disease}</title>
    ''' 


    return text,cr_info
        

In [34]:
_,info = parse_document(381)

In [35]:
print(info)


<id>381</id>:
<title>Бронхит</title>
    


In [36]:
documents_info = {}

In [37]:
for d in documents:
    text,info = parse_document(d)
    with open(f'data/KR_{d}.txt','w+',encoding='utf-8') as f:
        f.write(text)
    documents_info[d] = info
    print(f'{d} done')
with open('data/documents_info.json','w+',encoding='utf-8') as f:
    json.dump(documents_info,f)
print('json info dump done')

25 done
Related text was not found. Using content
261 done
306 done
313 done
352 done
359 done
360 done
381 done
603 done
654 done
655 done
656 done
662 done
664 done
677 done
683 done
714 done
724 done
749 done
783 done
json info dump done


In [38]:
print(documents_info['654'])


<id>654</id>:
<title>Внебольничная пневмония у взрослых</title>
    


In [3]:
url = f'https://apicr.minzdrav.gov.ru/api.ashx?op=GetClinrec2&id={381}&ssid=undefined'
request = requests.get(url)

data = request.json()['obj']['sections']

In [8]:
content = data[find_by_element_by_id(data,'doc_crat_info_1_4')]

In [10]:
soup = bs4.BeautifulSoup(content['content'])

In [11]:
print(soup.getText())

Острый бронхит (J20)
J20.0 — Острый бронхит, вызванный Mycoplasma pneumoniae
J20.1 — Острый бронхит, вызванный Haemophilus influenzae (палочкой Афанасьева-Пфейффера)
J20.2 — Острый бронхит, вызванный стрептококком
J20.3 — Острый бронхит, вызванный вирусом Коксаки
J20.4 — Острый бронхит, вызванный вирусом парагриппа
J20.5 — Острый бронхит, вызванный респираторным синцитиальным вирусом
J20.6 — Острый бронхит, вызванный риновирусом
J20.7 — Острый бронхит, вызванный эховирусом
J20.8 — Острый бронхит, вызванный другими уточненными агентами
J20.9 — Острый бронхит неуточненный
J40 — Бронхит, не уточненный как острый или хронический
J41 — Простой и слизисто-гнойный хронический бронхит
J42 — Хронический бронхит неуточненный


In [181]:
print(find_diseases_in_content(data))

J30.1 – Аллергический ринит, вызванный пыльцой растений
J30.2 – Другие сезонные аллергические риниты
J30.3 – Другие аллергические риниты
J30.4 – Аллергический ринит неуточненный

